In [1]:
import os

from os.path import join as ospj
from os.path import expanduser
from munch import Munch as mch
import numpy as np

from ProbVLM.src.ds import prepare_coco_dataloaders, prepare_flickr_dataloaders, prepare_cub_dataloaders, prepare_flo_dataloaders

from utils import *
from ProbVLM.src.networks import *
from ProbVLM.src.train_probVLM import *

import matplotlib.pyplot as plt


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/Inter-CLIP/Inter-ProbVLM/ProbVLM/src/train_probVLM.py:26: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:83.)
  dtype=torch.cuda.FloatTensor(),


In [2]:
dataset = 'coco' # coco or flickr
data_dir = ospj('dataset', dataset) # e.g. ospj(expanduser('~'), 'Documents', 'jm', 'data', dataset)
dataloader_config = mch({
    'batch_size': 32,
    'random_erasing_prob': 0.,
    'traindata_shuffle': True
})
loaders = load_data_loader(dataset, data_dir, dataloader_config, num_workers=12)
coco_train_loader, coco_valid_loader, coco_test_loader = loaders['train'], loaders['val'], loaders['test']

creating index...
index created!
length of dataset:  566435
Loading COCO Caption: n_images 113287 n_captions 566435...
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
length of dataset:  5000
Loading COCO Caption: n_images 1000 n_captions 5000...
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
length of dataset:  25000
Loading COCO Caption: n_images 5000 n_captions 25000...


In [3]:
test_batch = next(iter(coco_test_loader))
print(test_batch[0].shape)

torch.Size([32, 3, 224, 224])


In [4]:
# clip_net = load_model('cuda')
CLIP_Net = load_model(device='cuda', model_path=None)
ProbVLM_Net = BayesCap_for_CLIP(
    inp_dim=512,
    out_dim=512,
    hid_dim=256,
    num_layers=3,
    p_drop=0.05,
)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 4 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [5]:

# test_image = torch.rand(16, 3, 224, 224).cuda()
# test_text = torch.randint(0, 10000, (16, 77)).cuda()
# print(test_image, test_text)
# # load clip model
# clip_model = clip.load('ViT-B/32')[0].cuda()
# clip_model.eval()


# xfI, xfT = clip_model(test_image, test_text)
# print(xfI.shape, xfT.shape)

# image_features = clip_model.encode_image(test_image)
# text_features = clip_model.encode_text(test_text)
# print(image_features.shape, text_features.shape)

In [7]:
CLIP_Net = CLIP_Net.module if isinstance(CLIP_Net, torch.nn.DataParallel) else CLIP_Net
train_ProbVLM(
    CLIP_Net,
    ProbVLM_Net,
    coco_train_loader,
    coco_valid_loader,
    Cri = TempCombLoss(),
    device='cuda:2',
    dtype=torch.cuda.FloatTensor,
    init_lr=8e-5,
    num_epochs=500,
    eval_every=5,
    ckpt_path='exp/',
    T1=1e0,
    T2=1e-4,
    save_name='coco_probVLM'
)

  0%|          | 0/17702 [00:00<?, ?batch/s]

Epoch 0: 100%|██████████| 17702/17702 [07:19<00:00, 40.25batch/s, loss=0.051] 


Avg. loss: 0.08370770773321218


Validating ...: 100%|██████████| 157/157 [00:04<00:00, 31.74batch/s]


Avg. MSE: 0.000763863674364984 | Avg. MAE: 0.025264715775847435
current score: 0.025264715775847435 | Last best score: 100000000.0


...

Epoch 47: 100%|██████████| 17702/17702 [07:22<00:00, 40.03batch/s, loss=0.0196]


Avg. loss: 0.019300562282661176


Epoch 48: 100%|██████████| 17702/17702 [07:19<00:00, 40.24batch/s, loss=0.0197]


Avg. loss: 0.019295438417329173


Epoch 49:  17%|█▋        | 2973/17702 [01:16<05:52, 41.81batch/s, loss=0.0193]

In [ ]:
from one_agent import OneAgent
agent = OneAgent(agent_name='A')
agent = agent.cuda()
agent.eval()

OneAgent(
  (ProbVLM_Net): BayesCap_for_CLIP(
    (img_BayesCap): BayesCap_MLP(
      (mod): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): ReLU()
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): ReLU()
        (4): Dropout(p=0.05, inplace=False)
        (5): Linear(in_features=256, out_features=512, bias=True)
      )
      (block_mu): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
      )
      (block_alpha): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
      )
      (block_beta): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): ReLU()
      )
    )
    

In [ ]:
agent.communication_field_setup(coco_test_loader, coco_test_loader)

print(agent.w[:10])
print(agent.w.shape)

communication_field_setup
finish communication_field_setup
tensor([[49406,   320,   786,   593,   320,   736, 11122,   525,   320,  2442,
           617,  2966,   525,   320, 11795,  1759,   269, 49407,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [49406,   786,  6765,   320,  7659,  3701,   525,   320, 11795,  1759,
           525,   518, 16303,   269, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             

In [ ]:
CLIP_Net = CLIP_Net.module if isinstance(CLIP_Net, torch.nn.DataParallel) else CLIP_Net
train_ProbVLM(
    CLIP_Net,
    ProbVLM_Net,
    coco_test_loader,
    coco_valid_loader,
    Cri = TempCombLoss(),
    device='cuda',
    dtype=torch.cuda.FloatTensor,
    init_lr=8e-5,
    num_epochs=500,
    eval_every=5,
    ckpt_path='models/',
    T1=1e0,
    T2=1e-4,
    save_name='agent_A',
    w = agent.w
)

  0%|          | 0/782 [00:00<?, ?batch/s]

Epoch 0:  27%|██▋       | 212/782 [00:26<01:11,  7.95batch/s, loss=0.307]


KeyboardInterrupt: 